# Identification of enhancers case study 

This section will present a comparative analysis to demonstrate the application and performance of proPythia for addressing sequence-based prediction problems.

We'll try to replicate one of the [Ernest Bonat](https://medium.com/@ernest-bonat) [articles](https://medium.com/mlearning-ai/apply-machine-learning-algorithms-for-genomics-data-classification-132972933723#4536).

In [147]:
%load_ext autoreload

import sys
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, matthews_corrcoef
from sklearn.preprocessing import StandardScaler

sys.path.append('../../../../src/')
from propythia.shallow_ml import ShallowML

from descriptors import DNADescriptor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [148]:
dataset = pd.read_csv("dataset.csv")
print(dataset.shape)
print(dataset.head())

(2000, 2)
                                            sequence  label
0  CCGAGGGCTATGGTTTGGAAGTTAGAACCCTGGGGCTTCTCGCGGA...      0
1  GAGTTTATATGGCGCGAGCCTAGTGGTTTTTGTACTTGTTTGTCGC...      0
2  GATCAGTAGGGAAACAAACAGAGGGCCCAGCCACATCTAGCAGGTA...      0
3  GTCCACGACCGAACTCCCACCTTGACCGCAGAGGTACCACCAGAGC...      1
4  GGCGACCGAACTCCAACTAGAACCTGCATAACTGGCCTGGGAGATA...      1


In [149]:
# count number of each class
print(dataset.groupby('label').size())

label
0    1013
1     987
dtype: int64


To calculate features, and to be more easy, we create a function to calculate features, calculating all available DNA features.

In [150]:
# List of descriptors to be calculed. If empty, all descriptors will be calculated.
specifics = []

In [151]:
def calculate_feature(data):
    list_feature = []
    count = 0
    for seq in data['sequence']:
        res = {'sequence': seq}
        dna = DNADescriptor(seq)
        feature = dna.get_descriptors(specifics=specifics)
        res.update(feature)
        list_feature.append(res)
        
        # print progress every 100 sequences
        if count % 100 == 0:
            print(count, '/', len(data))

        count += 1
    print("Done!")
    df = pd.DataFrame(list_feature)
    return df

features = calculate_feature(dataset)

0 / 2000
100 / 2000
200 / 2000
300 / 2000
400 / 2000
500 / 2000
600 / 2000
700 / 2000
800 / 2000
900 / 2000
1000 / 2000
1100 / 2000
1200 / 2000
1300 / 2000
1400 / 2000
1500 / 2000
1600 / 2000
1700 / 2000
1800 / 2000
1900 / 2000
Done!


- In the dataframe, each row is a sequence and each column is a feature.
- There are 19 different features for each sequence.

In [152]:
fps_y = dataset['label']
fps_x = features.loc[:, features.columns != 'label']
fps_x = fps_x.loc[:, fps_x.columns != 'sequence']

In [153]:
no_need_normalization = ["length", "at_content", "gc_content"]

need_dict_normalization = ["nucleic_acid_composition", "enhanced_nucleic_acid_composition","dinucleotide_composition","trinucleotide_composition","k_spaced_nucleic_acid_pairs","kmer","PseDNC", "PseKNC"]

need_list_normalization = ["nucleotide_chemical_property", "accumulated_nucleotide_frequency", "DAC", "DCC", "DACC", "TAC","TCC","TACC", "binary"]

def normalize_dict(d, field):
    df = pd.json_normalize(d)
    df.columns = [str(field) + "_" + str(i) for i in df.columns]
    
    for f in df.columns:
        if isinstance(df[f][0], dict):
            df = pd.concat([df, normalize_dict(df[f], f)], axis=1)
            df.drop(f, axis=1, inplace=True)
    return df

def normalize_list(l, field):
    df = pd.DataFrame(l.to_list())
    df.columns = [str(field) + "_" + str(i) for i in df.columns]
    
    for f in df.columns:
        if isinstance(df[f][0], list):
            df = pd.concat([df, normalize_list(df[f], f)], axis=1)
            df.drop(f, axis=1, inplace=True)
    return df

new_fps_x = pd.DataFrame()

for col in fps_x.columns:
    if col in need_dict_normalization:
        new_fps_x = pd.concat([new_fps_x, normalize_dict(fps_x[col], col)], axis=1)
    elif col in need_list_normalization:
        new_fps_x = pd.concat([new_fps_x, normalize_list(fps_x[col], col)], axis=1)
    else:
        new_fps_x[col] = fps_x[col].to_numpy()
        
new_fps_x

,length,gc_content,at_content,nucleic_acid_composition_A,nucleic_acid_composition_C,nucleic_acid_composition_G,nucleic_acid_composition_T,enhanced_nucleic_acid_composition_0_A,enhanced_nucleic_acid_composition_0_C,enhanced_nucleic_acid_composition_0_G,...,binary_47_2,binary_47_3,binary_48_0,binary_48_1,binary_48_2,binary_48_3,binary_49_0,binary_49_1,binary_49_2,binary_49_3
0,50,0.60,0.40,0.18,0.26,0.34,0.22,0.2,0.4,0.4,...,0,0,0,1,0,0,0,1,0,0
1,50,0.48,0.52,0.12,0.16,0.32,0.40,0.2,0.0,0.4,...,0,1,0,1,0,0,0,0,1,0
2,50,0.54,0.46,0.34,0.26,0.28,0.12,0.4,0.2,0.2,...,0,0,0,1,0,0,0,0,0,1
3,50,0.64,0.36,0.24,0.42,0.22,0.12,0.2,0.4,0.2,...,0,0,0,0,0,1,0,0,1,0
4,50,0.54,0.46,0.28,0.26,0.28,0.18,0.2,0.2,0.6,...,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,50,0.66,0.34,0.16,0.26,0.40,0.18,0.0,0.4,0.4,...,1,0,1,0,0,0,0,0,1,0
1996,50,0.44,0.56,0.22,0.22,0.22,0.34,0.0,0.2,0.4,...,0,1,0,1,0,0,0,0,0,1
1997,50,0.46,0.54,0.18,0.24,0.22,0.36,0.2,0.4,0.2,...,0,0,0,1,0,0,0,0,0,1
1998,50,0.48,0.52,0.28,0.24,0.24,0.24,0.4,0.2,0.2,...,1,0,1,0,0,0,1,0,0,0


In [154]:
X_train, X_test, y_train, y_test = train_test_split(new_fps_x, fps_y, stratify=fps_y)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

ml = ShallowML(X_train, X_test, y_train, y_test, report_name=None, columns_names=new_fps_x.columns)

param_grid = [{'clf__n_estimators': [100, 250, 500, 750], 'clf__max_features': ['sqrt']}]

best_rf_model_enhancers = ml.train_best_model(
    model_name=None,
    model='rf',
    score=make_scorer(matthews_corrcoef),
    param_grid=param_grid,
    cv=10
)

performing gridSearch...
GridSearchCV took 10.68 seconds for 4 candidate parameter settings.
GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scl', None),
                                       ('clf',
                                        RandomForestClassifier(random_state=1))]),
             n_jobs=10,
             param_grid=[{'clf__max_features': ['sqrt'],
                          'clf__n_estimators': [100, 250, 500, 750]}],
             scoring=make_scorer(matthews_corrcoef))
Model with rank: 1
 Mean validation score: 0.987 (std: 0.010)
 Parameters: {'clf__max_features': 'sqrt', 'clf__n_estimators': 100}
 

Model with rank: 1
 Mean validation score: 0.987 (std: 0.010)
 Parameters: {'clf__max_features': 'sqrt', 'clf__n_estimators': 250}
 

Model with rank: 1
 Mean validation score: 0.987 (std: 0.010)
 Parameters: {'clf__max_features': 'sqrt', 'clf__n_estimators': 500}
 

Model with rank: 1
 Mean validation score: 0.987 (std: 0.010)
 Parameters: {'clf__max_features':

In [155]:
scores, report, cm, cm2 = ml.score_testset(best_rf_model_enhancers)
print(report)
print(cm)  
scores

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       253
           1       0.99      1.00      0.99       247

    accuracy                           0.99       500
   macro avg       0.99      0.99      0.99       500
weighted avg       0.99      0.99      0.99       500

[[250   3]
 [  0 247]]


{'Accuracy': 0.994,
 'MCC': 0.98807114368361,
 'log_loss': 0.09816478255252511,
 'f1 score': 0.993963782696177,
 'roc_auc': 0.9940711462450592,
 'Precision': array([0.988, 1.   ]),
 'Recall': array([1., 0.]),
 'fdr': 0.012,
 'sn': 1.0,
 'sp': 0.9881422924901185}